In [ ]:
import pandas as pd

imdb_file = 'IMDB Dataset.csv'
df=pd.read_csv(imdb_file)

In [ ]:
print(f'taille de mon data frame {df.shape}')
print(f'nom des colonnes {list(df.keys())}')

In [ ]:
print(df.head(5))

In [ ]:
row=3
print(df['review'][row])
print(df['sentiment'][row])

## nettoyer les balises html

In [ ]:
from preprocessing_tools import remove_html_tags

clean_text=remove_html_tags(df['review'][row])
print("AVANT NETTOYAGE:")
print(df['review'][row])
print("APRES NETTOYAGE:")
print(clean_text)

In [ ]:
df["review"]=df["review"].apply(lambda x:remove_html_tags(x))

In [ ]:
df.tail(5)

In [ ]:
sentiment_map={'positive':[1,0], 'negative':[0,1]}
print(sentiment_map['positive'])
print(sentiment_map['negative'])
df["sentiment"]=df["sentiment"].apply(lambda x:sentiment_map[x])
df.head()

## Séparer la base de données en ensemble d'apprentissage et de test

In [ ]:
import numpy as np

train_rate=0.80
dataset_size=df.shape[0]
train_size=int(train_rate*dataset_size)
print(f"Taille de la base de données {dataset_size} de l'ensemble d'apprentissage {train_size} et de test {dataset_size-train_size}")

# Tirer au hasard les indices de train
train_indices=np.random.choice(dataset_size,train_size, replace=False)
train_indices=sorted(train_indices)

# Collecter dans les indices de test les indices qui ne sont pas dans le train
train_indices_dict = {n:n for n in train_indices}
test_indices=[]
for n in range(dataset_size):
    try:
        train_indices_dict[n]
    except:
        test_indices.append(n)
        
df_train=df.iloc[train_indices]
df_test=df.iloc[test_indices]

print(f"Taille de l'ensemble de train {df_train.shape[0]}")
print(f"Taille de l'ensemble de test {df_test.shape[0]}")

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(df.review,df.sentiment,test_size=0.2)

## Pretraiter les review pour les transformer en entier

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

num_words = 20000
oov_token = '<unk>'
max_length=1000
pad_type='post'
trunc_type='post'
tokenizer = Tokenizer(num_words=num_words,oov_token=oov_token)

In [ ]:
tokenizer.__dict__.keys()

In [ ]:
train_reviews=list(x_train)
tokenizer.fit_on_texts(train_reviews)
train_sequences=tokenizer.texts_to_sequences(train_reviews)
train_padded=pad_sequences(train_sequences,padding=pad_type,truncating=trunc_type,maxlen=max_length)
train_labels=list(y_train)

test_reviews=list(x_test)
test_sequences=tokenizer.texts_to_sequences(test_reviews)
test_padded=pad_sequences(test_sequences,padding=pad_type,truncating=trunc_type,maxlen=max_length)
test_labels=list(y_test)

## Implementer le classifieur avec tensorflow keras

In [ ]:
#Implement the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding

def create_model_seq_api(vocab_size,embedding_dim,hidden_layer_dims,max_length):
    """
    create model using sequential API
    """
    model = Sequential()
    model.add(Embedding(vocab_size,embedding_dim,input_length=max_length))
    model.add(Flatten())
    
    for dim in hidden_layer_dims:
        model.add(Dense(dim,activation='relu'))
    
    model.add(Dense(2, activation='softmax'))
    return model

In [ ]:
#Create the model
vocab_size=len(tokenizer.word_index)
word_embedding_dim=16
hidden_layer_dims=[100]

model=create_model_seq_api(vocab_size=vocab_size,
                           embedding_dim=word_embedding_dim,
                           hidden_layer_dims=hidden_layer_dims,
                           max_length=max_length)
print(model.summary())

In [ ]:
#Train the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

summary=model.fit(train_padded,np.array(train_labels),epochs=2,verbose=1)

In [ ]:
#Evaluate the model
loss, accuracy = model.evaluate(test_padded,np.array(test_labels),verbose=1)
print('Accuracy: %f' % (accuracy*100))

In [1]:
prediction=model.predict([test_padded[0]])
print(prediction)

NameError: name 'model' is not defined